## Case classes and pattern matching

### Case classes versus ordinary classes

* what are case classes
* what are they good for

In [1]:
class Person(val name: String, age: Int, socialSecurityNumber: Long) {
  import Person._
    
  val isPensioner: Boolean = age >= 65
    
  lazy val hasValidSSN: Boolean = longLastingQueryOfDatabase(socialSecurityNumber)
}

object Person {
  def longLastingQueryOfDatabase(ssn: Long): Boolean = {
    // insert long code here
    ???
  }
}

defined class Person
defined object Person

In [2]:
val p1 = new Person("John", 50, 123456789L)
val p2 = new Person("John", 50, 123456789L)
println(p1)
println(p2 == p1)
println(Set(p1, p2).size)

$sess.cmd0Wrapper$Helper$Person@7f53ad04
false
2


p1: Person = $sess.cmd0Wrapper$Helper$Person@7f53ad04
p2: Person = $sess.cmd0Wrapper$Helper$Person@60c1248c

In [3]:
case class Person(name: String, age: Int, socialSecurityNumber: Long) {
  import Person._
    
  val isPensioner: Boolean = age >= 65
    
  lazy val isValidSSN: Boolean = longLastringQueryOfDatabase(socialSecurityNumber)
}

object Person {
  def longLastringQueryOfDatabase(ssn: Long): Boolean = ???
}

defined class Person
defined object Person

In [4]:
val p1 = Person("John", 50, 123456789L)
val p2 = Person(name = "John", age = 50, socialSecurityNumber = 123456789L)
println(p1)
println(p1 == p2)
println(Set(p1, p2).size)

Person(John,50,123456789)
true
1


p1: Person = Person("John", 50, 123456789L)
p2: Person = Person("John", 50, 123456789L)

Case classes can be seen as immutable data-storing objects that  depend on their constructor arguments only. Those arguments are automatically become fields / methods of the class.

This functional concept allows us to
* use a compact initialisation syntax (without the `new` keyword)
* decompose them using **pattern matching**
* have equality comparisons and toString method defined

If a class performs stateful computations inside or exhibits other kinds of complex behaviour, it should be an ordinary class.

In [5]:
sealed abstract class Tree {
  def isEmpty: Boolean
}

case class Node(value: Int, left: Tree, right: Tree) extends Tree {
  def isEmpty: Boolean = false
  
  override def toString: String = value + left.toString + right.toString // preorder traversal
}

case object Empty extends Tree {
  def isEmpty: Boolean = true
  
  override def toString: String = "."
}

defined class Tree
defined class Node
defined object Empty

In [6]:
val myTree: Tree = Node(5, 
                        Node(4, 
                             Node(1, Empty, Empty), 
                             Node(2, Empty, Empty)), 
                        Node(6, Empty, Empty))
println(myTree)

541..2..6..


myTree: Tree = 541..2..6..

### Pattern matching

* syntax: selector `match` { alternatives }
* match is a `switch` on steroids
* match is an expression, that is, it always results in a value
* it does not fall-through
* if none of the alternatives match, an MatchError exception will be thrown

#### Constant patterns

Any literal can be used as a pattern. A constant pattern matches only with itself.

In [7]:
def writeOutNumbers(x: Int): String = x match {
  case 0 => "zero"
  case 1 => "one"
  case 2 => "two"
  case _ => "many (or maybe negative?)"  // wildcard pattern
}

defined function writeOutNumbers

In [8]:
println(writeOutNumbers(1))
println(writeOutNumbers(-3))

one
many (or maybe negative?)


#### Variable patterns

A variable pattern matches with any object, just like a wildcard!

In [9]:
val hello: String = "Hello"
val ahoy: String = "Ahoy"

"Ahoy" match {
  case hello => println("You have made a mistake")
  case ahoy => println("Yes, it is")
  case _ => println("no idea what's going on")
}

You have made a mistake


hello: String = "Hello"
ahoy: String = "Ahoy"

In [10]:
"Ahoy" match {
  case `hello` => println("You have made a mistake")
  case `ahoy` => println("Yes, it is")
  case _ => println("no idea what's going on")
}

Yes, it is


#### Constructor patterns

This provides deep matches inside case classes.

To show this, let's define a nested hierarchy of case classes and objects.

In [11]:
sealed abstract class ProgrammingLanguage
case object Scala extends ProgrammingLanguage
case object Python extends ProgrammingLanguage
case object COBOL extends ProgrammingLanguage
case object Haskell extends ProgrammingLanguage
case object Clojure extends ProgrammingLanguage

defined class ProgrammingLanguage
defined object Scala
defined object Python
defined object COBOL
defined object Haskell
defined object Clojure

In [12]:
sealed abstract class Employee {
  val id: Int
  val name: String
  def greet: String = s"Hello, $name!"
}

case class CEO(id: Int, name: String, salary: Long) extends Employee {
  override def greet: String = "Yes, Master!"
}

case class Engineer(id: Int,
                    name: String,
                    salary: Int,
                    language: ProgrammingLanguage) extends Employee

case class Trainee(id: Int, name: String, salary: Int, boss: Engineer) extends Employee {
  override def greet: String = "Hello, have you finished your task?"
}

defined class Employee
defined class CEO
defined class Engineer
defined class Trainee

In [13]:
val joe: Engineer = Engineer(id = 11, name = "Joe", salary = 60000, language = Haskell)
val bill: Employee = Trainee(id = 101, name = "Bill", salary = 10000, boss = joe)

joe: Engineer = Engineer(11,Joe,60000,Haskell)
bill: Employee = Trainee(101,Bill,10000,Engineer(11,Joe,60000,Haskell))

In [14]:
bill match {
  case Trainee(_, "Adam", _, joe) => println("Bill is called Adam???")
  case Engineer(_, _, _, _) => println("Bill is an engineer???") // there is a better way to match type
  case Trainee(_, _, _, Engineer(_, _, _, Haskell)) => println("Bill's boss knows Haskell!!")
  case Trainee(_, "Bill", _, _) => println("Bill is Bill indeed.")
  case _ => println("No one knows who this guy is.")
}

Bill's boss knows Haskell!!


#### Other patterns

* sequence patterns
* typed patterns
* variable binding inside a pattern match

In [15]:
val list: List[Int] = List(1, 1, 3, 5, 8, 13)

list match {
  case Nil => println("The list is empty.")
  case x :: xs => println(s"The head element is $x")
  // x :: xs means that the list has head element x and tail xs
}

list match {
  case Nil => println("The list is empty.")
  case List(1, 2, 3, _*) => println("This list has length at least 3 starting with 1, 2 and 3.")
  case List(_, _, 3, _*) => println("The 3rd element is 3.")
  case _ => println("no match")
}

The head element is 1
The 3rd element is 3.


list: List[Int] = List(1, 1, 3, 5, 8, 13)

In [16]:
bill match {
  case e: Engineer => println("Bill is an engineer?")
  case t: Trainee => println("Bill is a trainee.")
  case _ => println("Maybe he is the new CEO?")
}

Bill is a trainee.


In [17]:
bill match {
  case Trainee(_, "Adam", _, joe) => println("Bill is called Adam???")
  case Engineer(_, _, _, _) => println("Bill is an engineer???") // there is a better way to match type
  case Trainee(_, _, _, billsBoss @ Engineer(_, _, _, Haskell)) =>
    println(s"Bill's boss is $billsBoss!!")
  case Trainee(_, "Bill", _, _) => println("Bill is Bill indeed.")
  case _ => println("No one knows who this guy is.")
}

Bill's boss is Engineer(11,Joe,60000,Haskell)!!


#### Pattern matches everywhere

In [ ]:
val Trainee(id, _, amount, _) = bill
println(id)
println(amount)

val (a: Int, b: Int) = (0, 1)
println(a)
println(b)

// and the list goes on and on ...

But there are patterns that cannot be matched. Scala has type erasure, no information about type arguments is maintained at runtime.

In [19]:
val lst: List[Int] = List(1, 2, 5, 8)

lst match {
  case a: List[String] => println("this is a StringList")
  case b: List[Int] => println("this is an IntList")
}

this is a StringList


lst: List[Int] = List(1, 2, 5, 8)